# Building RAG with Qwen2.5

In [ ]:
import json
import os
import shutil
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Step 0: Clear the Chroma database if it exists
persist_dir = "./chroma.db"
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)

# Step 1: Folder for JSON Files
input_folder = "Vietnam-Law-rag_json"

# Step 2: Load All JSON Files and Convert to LangChain Documents
documents = []
for file_name in os.listdir(input_folder):
    if file_name.endswith(".json"):
        file_path = os.path.join(input_folder, file_name)
        base_file_name = os.path.splitext(file_name)[0]  # Remove the extension for `file_id`
        
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        
        # Convert JSON data to LangChain Document objects
        documents.extend([
            Document(
                page_content=entry["text"],
                metadata={
                    "id": entry["id"],
                    "article": entry["article"],
                    "clause": entry["clause"],
                    "title": entry["title"],
                    "file_id": base_file_name
                }
            )
            for entry in data
        ])

print(f"Loaded {len(documents)} documents from {input_folder}.")

# Step 3: Initialize HuggingFace Embeddings
embeddings_model = HuggingFaceEmbeddings()

# Step 4: Create Chroma Vector Store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=persist_dir
)

print("Chroma database created and saved at:", persist_dir)

# Test query
## RAG database builded on cloud servers, fetch them then run the below cell

The aim is to optimize the returned data after the query search before push into the LLM Models, below here use Qwen2.5 for example.

Just download the chroma.db, then symlink or put them in the current working git folder, then run the second cell.

### CELL 2

In [1]:
import torch
from langchain.vectorstores import Chroma
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings

# Step 1: Load the Chroma Database
persist_dir = "./chroma.db"

# Initialize the embedding function
embeddings_model = HuggingFaceEmbeddings()

# Load the Chroma database with the embedding function
vectorstore = Chroma(
    persist_directory=persist_dir,
    embedding_function=embeddings_model
)

print("Chroma database loaded.")

# Step 2: Load Qwen Model
model_id = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Step 3: Set Device for GPU/CPU
device = 0 if torch.cuda.is_available() else -1

# Step 4: Create a Text-Generation Pipeline with GPU/CPU
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    device=device,
    clean_up_tokenization_spaces=True
)

# Wrap the pipeline for LangChain
hf = HuggingFacePipeline(pipeline=pipe)

print("Model and pipeline initialized.")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

/tmp/ipykernel_741/1845614436.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Chroma database loaded.
Model and pipeline initialized.


### CELL 3

In [8]:
import re
from langchain.chains import RetrievalQA

# Move model to GPU
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Increase retrieval limit
retriever = vectorstore.as_retriever(search_kwargs={"k": 300})

qa_chain = RetrievalQA.from_chain_type(
    llm=hf,
    retriever=retriever,
    return_source_documents=True
)

# Query and retrieval
query = "Trẻ em là gì"
result = qa_chain({"query": query})

# Print the Result
print("Answer:", result["result"])

# Print the Source Documents
print("Source Documents:")
for doc in result["source_documents"]:
    print(f"Metadata: {doc.metadata}")
    print(f"Content: {doc.page_content}\n")

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Trẻ em bị bỏ rơi;

Trẻ em.

Trạm y tế;

Quản trị rủi ro;

Trẻ em mồ côi cả cha và mẹ;

Trẻ em bị mua bán;

Tạm giữ tàu biển;

Cơ quan quản lý nhà nước về trẻ em;

Cơ quan quản lý nhà nước về trẻ em;

Cơ quan quản lý nhà nước về trẻ em;

Cơ quan quản lý nhà nước về trẻ em;

Cơ quan quản lý nhà nước về trẻ em;

Cướp biển, cướp có vũ trang;

Quê quán.

Cơ sở y tế;

Cơ sở y tế.

Cơ sở y tế.

Quê quán;

Quê quán;

Quê quán;

Có giáo lý, giáo luật, lễ nghi;

Quản chế;

Hỗ trợ y tế;

Biện pháp sơ cứu về y tế;

Y tế, văn hóa, giáo dục;

Cột cây số;

Trẻ em bị bóc lột;

Viện pháp y quốc gia thuộc Bộ Y tế;

Trẻ em vi phạm pháp luật;

Chi viện trợ.

Có giáo lý, giáo luật;

Diễn biến của quá trình hòa giải;

Hồ sơ mời thầu.

Bị can có nghĩa vụ:

Hỗ trợ tâm lý;

Cơ sở dữ liệu về con người;

Tên tàu biển;

Trẻ em của cơ sở giáo dụ

### CELL 4

In [7]:
import re
from langchain.chains import RetrievalQA
import torch

# Set const k_retrieval as 300
k_retrieval = 300

# Move model to GPU
torch.cuda.empty_cache()
model.to("cuda" if torch.cuda.is_available() else "cpu")

retriever = vectorstore.as_retriever(search_kwargs={"k": k_retrieval})

# Define a custom retriever function that retrieves documents without filtering
def custom_retriever(query, k=k_retrieval):
    # Retrieve relevant documents from the retriever
    results = retriever.get_relevant_documents(query)
    
    # Directly return the retrieved documents without filtering or modification
    return results

# Define the instruction for the model
instruction = "Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

# Use the custom retriever to retrieve and modify documents
query = "Trẻ em là gì"
retrieved_docs = custom_retriever(query)

# Debug: Check if any documents are retrieved and modified
if not retrieved_docs:
    print("No relevant documents retrieved or filtered out.")
else:
    print(f"Retrieved {len(retrieved_docs)} documents.")

# Concatenate the instruction with the retrieved documents
input_to_llm = instruction + " " + " ".join([doc.page_content for doc in retrieved_docs])

# Check the input format before passing to the model
print("Input to LLM:", input_to_llm[:500])  # Print the first 500 characters for debugging

# Send the modified input to your LLM (e.g., Qwen2.5 model)
result_from_llm = hf(input_to_llm)  # Assuming 'hf' is your model callable
print("LLM Output:", result_from_llm)

# Move model to CPU to release GPU memory
torch.cuda.empty_cache()
model.to("cpu")
torch.cuda.empty_cache()

Retrieved 300 documents.
Input to LLM: Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Trẻ em bị bỏ rơi; Trẻ em. Trạm y tế; Quản trị rủi ro; Trẻ em mồ côi cả cha và mẹ; Trẻ em bị mua bán; Tạm giữ tàu biển; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cướp biển, cướp có vũ trang; Q
LLM Output: Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Trẻ em bị bỏ rơi; Trẻ em. Trạm y tế; Quản trị rủi ro; Trẻ em mồ côi cả cha và mẹ; Trẻ em bị mua bán; Tạm giữ tàu biển; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản lý nhà nước về trẻ em; Cơ quan quản l